<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/DZ_RNN2_v3_%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
'''Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y(0) = x(0), y(i) = x(i) + x(0). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU'''

'Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)\nи задавались следующим образом:\nx - последовательность цифр\ny(0) = x(0), y(i) = x(i) + x(0). Если y(i) >= 10, то y(i) = y(i) - 10\n\nЗадача:\nнаучить модель предсказывать y(i) по x(i)\nпробовать RNN, LSTM, GRU'

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
x=[]
y=[]
y_sum=[]
max_str=10000

In [33]:
X_1 = torch.randint(0,9,(max_str,32))
x_np = X_1.numpy()

for i in range(max_str):
    x=x_np[i].tolist()

    y_vr=0
    y.append(x[0])
    for i in range(1,len(x)):
        y_vr = x[i]+x[0]
        if (y_vr < 10):
            y.append(y_vr)
        else:
            y.append(y_vr-10)

    y_sum.append(y)
    y=[]
tar=torch.from_numpy(np.array(y_sum))

In [34]:
X_1

tensor([[1, 3, 5,  ..., 7, 7, 4],
        [3, 1, 3,  ..., 1, 2, 5],
        [5, 5, 7,  ..., 7, 0, 4],
        ...,
        [3, 5, 0,  ..., 6, 1, 2],
        [8, 5, 7,  ..., 4, 1, 4],
        [6, 8, 2,  ..., 2, 5, 8]])

In [35]:
tar

tensor([[1, 4, 6,  ..., 8, 8, 5],
        [3, 4, 6,  ..., 4, 5, 8],
        [5, 0, 2,  ..., 2, 5, 9],
        ...,
        [3, 8, 3,  ..., 9, 4, 5],
        [8, 3, 5,  ..., 2, 9, 2],
        [6, 4, 8,  ..., 8, 1, 4]])

In [36]:
# Преобразование размерностей для RNN
X = X_1.unsqueeze(-1)            # (10000, 32, 1) - добавляем размерность признака
y = tar.unsqueeze(-1)            # (10000, 32, 1)

In [37]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X.numpy(), y.numpy(), test_size=0.2, random_state=42
)

In [38]:
# Конвертация в тензоры PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [39]:
# Создание Dataset и DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [40]:
# Определение модели GRU
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        output = self.fc(gru_out)
        return output

In [41]:
# Определение модели стандартная RNN
class RNNModel_RNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        output = self.fc(rnn_out)
        return output

In [42]:
# Определение модели LSTM
class RNNModel_LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return output

In [43]:
# Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNModel().to(device)
model_lstm = RNNModel_LSTM().to(device)
model_rnn = RNNModel_RNN().to(device)
print(model)
print(model_lstm)
print(model_rnn)

RNNModel(
  (gru): GRU(1, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)
RNNModel_LSTM(
  (lstm): LSTM(1, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)
RNNModel_RNN(
  (rnn): RNN(1, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)


In [44]:
# Функция потерь и оптимизатор
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=0.001)
optimizer_rnn = optim.Adam(model_rnn.parameters(), lr=0.001)

In [45]:
 # Обучение модели GRU
num_epochs = 50

for epoch in range(num_epochs):
    # Режим обучения
    model.train()
    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Прямой проход
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * batch_X.size(0)



In [46]:
# Валидация GRU
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item() * batch_X.size(0)

test_loss = test_loss / len(test_loader.dataset)

print(f'Epoch [{epoch+1}/{num_epochs}] | '
          f'Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}')



Epoch [50/50] | Train Loss: 37.456586 | Test Loss: 0.004660


In [47]:
 # Обучение модели LSTN
for epoch in range(num_epochs):
    # Режим обучения
    model_lstm.train()
    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Прямой проход
        outputs = model_lstm(batch_X)
        loss = criterion(outputs, batch_y)

        # Обратный проход и оптимизация
        optimizer_lstm.zero_grad()
        loss.backward()
        optimizer_lstm.step()

        train_loss += loss.item() * batch_X.size(0)


In [48]:
 # Обучение модели RNN
for epoch in range(num_epochs):
    # Режим обучения
    model_rnn.train()
    train_loss = 0.0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Прямой проход
        outputs = model_rnn(batch_X)
        loss = criterion(outputs, batch_y)

        # Обратный проход и оптимизация
        optimizer_rnn.zero_grad()
        loss.backward()
        optimizer_rnn.step()

        train_loss += loss.item() * batch_X.size(0)


In [49]:
# Валидация LSTM
model_lstm.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model_lstm(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item() * batch_X.size(0)

test_loss = test_loss / len(test_loader.dataset)

print(f'Модель с LSTM Epoch [{epoch+1}/{num_epochs}] | '
          f'Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}')

Модель с LSTM Epoch [50/50] | Train Loss: 47176.448425 | Test Loss: 0.089533


In [50]:
# Валидация RNN
model_rnn.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model_rnn(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item() * batch_X.size(0)

test_loss = test_loss / len(test_loader.dataset)

print(f'Модель с RNN Epoch [{epoch+1}/{num_epochs}] | '
          f'Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}')

Модель с RNN Epoch [50/50] | Train Loss: 47176.448425 | Test Loss: 5.815446


In [51]:
# Вычисление MSE для модели с GRU на всем тестовом наборе
model.eval()
with torch.no_grad():

    test_preds = model(X_test.to(device))
    final_loss = criterion(test_preds, y_test.to(device))
    print(f'\nFinal Test MSE для модели с GRU: {final_loss.item():.6f}')




Final Test MSE для модели с GRU: 0.004660


In [52]:
# Вычисление MSE для модели с LSTM на всем тестовом наборе
model_lstm.eval()
with torch.no_grad():

    test_preds = model_lstm(X_test.to(device))
    final_loss = criterion(test_preds, y_test.to(device))
    print(f'\nFinal Test MSE для модели с LSTM: {final_loss.item():.6f}')


Final Test MSE для модели с LSTM: 0.089533


In [53]:
# Вычисление MSE для модели с RNN на всем тестовом наборе
model_rnn.eval()
with torch.no_grad():

    test_preds = model_rnn(X_test.to(device))
    final_loss = criterion(test_preds, y_test.to(device))
    print(f'\nFinal Test MSE для модели с RNN: {final_loss.item():.6f}')


Final Test MSE для модели с RNN: 5.815445


In [54]:
# Предсказания для модели с GRU
sample = X_test[:1].to(device)
predictions = model(sample)
X_1=X_test[:1].numpy() #Исходная последовательность
predict_1=np.round(predictions.detach().numpy()) #предсказанная последовательность

result = np.c_[X_1,predict_1]
print(result)


[[[1. 1.]
  [3. 4.]
  [7. 8.]
  [2. 3.]
  [3. 4.]
  [0. 1.]
  [8. 9.]
  [2. 3.]
  [5. 6.]
  [6. 7.]
  [4. 5.]
  [0. 1.]
  [3. 4.]
  [7. 8.]
  [7. 8.]
  [5. 6.]
  [7. 8.]
  [1. 2.]
  [1. 2.]
  [2. 3.]
  [6. 7.]
  [6. 7.]
  [0. 1.]
  [1. 2.]
  [8. 9.]
  [7. 8.]
  [2. 3.]
  [5. 6.]
  [2. 3.]
  [5. 6.]
  [3. 4.]
  [8. 9.]]]


In [55]:
# Предсказания для модели с LSTM
sample = X_test[:1].to(device)
predictions = model_lstm(sample)
X_1=X_test[:1].numpy() #Исходная последовательность
predict_1=np.round(predictions.detach().numpy()) #предсказанная последовательность

result = np.c_[X_1,predict_1]
print(result)


[[[1. 1.]
  [3. 4.]
  [7. 8.]
  [2. 3.]
  [3. 4.]
  [0. 1.]
  [8. 9.]
  [2. 3.]
  [5. 6.]
  [6. 7.]
  [4. 5.]
  [0. 1.]
  [3. 4.]
  [7. 8.]
  [7. 8.]
  [5. 6.]
  [7. 8.]
  [1. 2.]
  [1. 2.]
  [2. 3.]
  [6. 7.]
  [6. 7.]
  [0. 1.]
  [1. 2.]
  [8. 9.]
  [7. 8.]
  [2. 3.]
  [5. 6.]
  [2. 3.]
  [5. 6.]
  [3. 4.]
  [8. 9.]]]


In [56]:
# Предсказания для модели с RNN
sample = X_test[:1].to(device)
predictions = model_rnn(sample)
X_1=X_test[:1].numpy() #Исходная последовательность
predict_1=np.round(predictions.detach().numpy()) #предсказанная последовательность

result = np.c_[X_1,predict_1]
print(result)


[[[1. 1.]
  [3. 4.]
  [7. 7.]
  [2. 3.]
  [3. 4.]
  [0. 1.]
  [8. 7.]
  [2. 3.]
  [5. 6.]
  [6. 7.]
  [4. 5.]
  [0. 1.]
  [3. 4.]
  [7. 7.]
  [7. 7.]
  [5. 6.]
  [7. 7.]
  [1. 2.]
  [1. 2.]
  [2. 3.]
  [6. 7.]
  [6. 7.]
  [0. 1.]
  [1. 2.]
  [8. 7.]
  [7. 7.]
  [2. 3.]
  [5. 6.]
  [2. 3.]
  [5. 6.]
  [3. 4.]
  [8. 7.]]]


In [ ]:
#Вывод: Для решения этой задачи подходят RNN c GRU и LSTM слоями, при этом MSE лучше у RNN с GRU и для достижения хорошего результата RNN с GRU требуется меньшее количество эпох (20).